# Forecasting dengan AutoKorelasi

## DataUnderstanding

### Import Library  
Kode ini berfungsi untuk mengimpor semua library Python yang diperlukan untuk analisis.

In [1]:
import openeo
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import xarray as xr

ModuleNotFoundError: No module named 'openeo'

### Koneksi ke Server  
Cell ini digunakan untuk membangun koneksi ke backend openeo dan melakukan otentikasi menggunakan protokol OIDC. Langkah ini diperlukan untuk mendapatkan akses resmi ke koleksi data yang tersedia di server.

In [2]:
# Connect to openEO backend
connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

Authenticated using refresh token.


### Penentuan Area dan Rentang Waktu  
Blok kode ini mendefinisikan parameter spasial (spatial extent) dan temporal (temporal extent) untuk akuisisi data. spatial_extent menentukan batas geografis area penelitian (Surabaya), sedangkan start_date dan end_date membatasi rentang waktu data yang akan diunduh.

In [5]:
# Area of Interest (AOI) - versi kamu
aoi = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              112.73333744680662,
              -7.213897744108465
            ],
            [
              112.73333744680662,
              -7.240796541868804
            ],
            [
              112.76429592850928,
              -7.240796541868804
            ],
            [
              112.76429592850928,
              -7.213897744108465
            ],
            [
              112.73333744680662,
              -7.213897744108465
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

# Define spatial extent from AOI coordinates (disesuaikan juga)
spatial_extent = {
    "west": 112.73333744680662,
    "south": -7.240796541868804,
    "east": 112.76429592850928,
    "north": -7.213897744108465
}

# Rentang waktu (kamu bisa ubah sesuai kebutuhan)
start_date = "2020-10-23"
end_date = "2025-10-23"

print(f" AOI defined for coordinates: {spatial_extent}")
print(f" Time range: {start_date} to {end_date}")
print(" Setup completed successfully")


 AOI defined for coordinates: {'west': 112.73333744680662, 'south': -7.240796541868804, 'east': 112.76429592850928, 'north': -7.213897744108465}
 Time range: 2020-10-23 to 2025-10-23
 Setup completed successfully


![tampilanMiniforge1](./images/kordinat.png)

### Pengambilan dan Agregasi Data
Kode ini menginstruksikan server untuk memuat koleksi data polusi NO2 dari satelit Sentinel-5P sesuai parameter yang telah ditentukan. Fungsi aggregate_temporal_period digunakan untuk merata-ratakan data harian menjadi satu nilai tunggal per hari (period="day").

In [ ]:
print("Loading Sentinel-5P NO2 data...")

s5p_no2 = connection.load_collection(
    "SENTINEL_5P_L2",
    temporal_extent=[start_date, end_date],
    spatial_extent=spatial_extent,
    bands=["NO2"],
)

s5p_monthly = s5p_no2.aggregate_temporal_period(
    period="day",
    reducer="mean"
)

print("Data collection and aggregation configured successfully")

Loading Sentinel-5P NO2 data...
Data collection and aggregation configured successfully


### Eksekusi Proses dan Pengunduhan
Fungsi execute_batch memulai proses pengumpulan dan agregasi data di sisi server secara asynchronous. Hasil dari proses ini akan disimpan dalam sebuah file output dengan format NetCDF (.nc).

In [7]:
print("Starting data processing job...")

job = s5p_monthly.execute_batch(
    title="NO2 Averages 2020-2025", 
    outputfile="no2_averages_4years.nc"
)


Starting data processing job...
0:00:00 Job 'j-25102316381049339dfcf6a3c1e3061a': send 'start'
0:00:30 Job 'j-25102316381049339dfcf6a3c1e3061a': created (progress 0%)
0:00:36 Job 'j-25102316381049339dfcf6a3c1e3061a': created (progress 0%)
0:00:43 Job 'j-25102316381049339dfcf6a3c1e3061a': created (progress 0%)
0:00:52 Job 'j-25102316381049339dfcf6a3c1e3061a': created (progress 0%)
0:01:03 Job 'j-25102316381049339dfcf6a3c1e3061a': running (progress N/A)
0:01:16 Job 'j-25102316381049339dfcf6a3c1e3061a': running (progress N/A)
0:01:32 Job 'j-25102316381049339dfcf6a3c1e3061a': running (progress N/A)
0:01:52 Job 'j-25102316381049339dfcf6a3c1e3061a': running (progress N/A)
0:02:19 Job 'j-25102316381049339dfcf6a3c1e3061a': running (progress N/A)
0:02:50 Job 'j-25102316381049339dfcf6a3c1e3061a': running (progress N/A)
0:03:30 Job 'j-25102316381049339dfcf6a3c1e3061a': running (progress N/A)
0:04:18 Job 'j-25102316381049339dfcf6a3c1e3061a': running (progress N/A)
0:05:17 Job 'j-25102316381049339d

### Konversi dari Format .nc ke .csv
Cell ini berfungsi untuk membaca file .nc yang telah diunduh menggunakan xarray, kemudian mengonversinya ke dalam format DataFrame pandas. Hasilnya diekspor menjadi file hasil-data.csv untuk pengolahan lebih lanjut yang lebih mudah.

In [3]:
# 1. Buka file .nc Anda
# Ganti 'openEO.nc' jika nama filenya berbeda
ds = xr.open_dataset("D:\VSCode1\MatKul-psd\PSD\MataKuliah-Psd\dataset\openEO.nc", engine="h5netcdf")

# 2. Konversi dataset xarray menjadi DataFrame pandas
df = ds.to_dataframe()

# 3. [PENTING] Ratakan Multi-Index
# Data NetCDF biasanya punya banyak index (spt waktu, lat, lon).
# 'reset_index()' akan mengubah semua index itu menjadi kolom biasa.
df_flat = df.reset_index()

# 4. Simpan sebagai file CSV
df_flat.to_csv('dataset/hasil-data.csv', index=False)

print("Selesai! File 'hasil-data.csv' telah dibuat.")

Selesai! File 'hasil-data.csv' telah dibuat.


## DataPreProcessing

### Pengurutan Data Berdasarkan Waktu
Kode ini melakukan pengurutan data secara kronologis berdasarkan kolom waktu ('t'). Ini adalah langkah preprocessing yang fundamental untuk data time-series guna memastikan urutan observasi sudah benar.

In [2]:
# 1. Baca file CSV
df = pd.read_csv('./dataset/hasil-data.csv')

# 2. Ubah kolom 't' menjadi format datetime
df['t'] = pd.to_datetime(df['t'])

# 3. Urutkan DataFrame berdasarkan kolom 't'
df_sorted = df.sort_values(by='t')

# 4. Menampilkan hasil pengirutan
print(df_sorted)

print("Selesai! Data telah diurutkan berdasarkan waktu dan disimpan di './dataset/hasil-data-sorted.csv'")

              t           x         y  crs       NO2
0    2020-10-23  112.760619 -7.223435  b''       NaN
1    2020-10-24  112.760619 -7.223435  b''  0.000034
2    2020-10-25  112.760619 -7.223435  b''  0.000012
3    2020-10-26  112.760619 -7.223435  b''       NaN
4    2020-10-27  112.760619 -7.223435  b''  0.000141
...         ...         ...       ...  ...       ...
1800 2025-10-18  112.760619 -7.223435  b''  0.000026
1801 2025-10-19  112.760619 -7.223435  b''       NaN
1802 2025-10-20  112.760619 -7.223435  b''       NaN
1803 2025-10-21  112.760619 -7.223435  b''       NaN
1804 2025-10-22  112.760619 -7.223435  b''       NaN

[1805 rows x 5 columns]
Selesai! Data telah diurutkan berdasarkan waktu dan disimpan di './dataset/hasil-data-sorted.csv'


### Interpolasi untuk Mengisi Data Hilang
Langkah ini bertujuan untuk menangani nilai yang hilang (NaN) dalam data. Metode interpolasi berbasis waktu (method='time') digunakan untuk mengisi kekosongan data dengan memperkirakan nilainya berdasarkan interval waktu antara titik data yang valid.

In [5]:
df_indexed = df_sorted.set_index('t')

# 5. Lakukan interpolasi pada kolom 'NO2'
df_indexed['NO2'] = df_indexed['NO2'].interpolate(method='time')

# 6. Kembalikan 't' menjadi kolom & simpan di variabel baru
df_interpolated = df_indexed.reset_index()


# --- Konfirmasi Hasil ---
print("Selesai! Data sudah diinterpolasi dan disimpan dalam variabel 'df_interpolated'.")
print("\nCuplikan 5 baris pertama dari data hasil interpolasi:")
print(df_interpolated.head())
print(f"\nSisa missing values di kolom 'NO2': {df_interpolated['NO2'].isna().sum()}")

Selesai! Data sudah diinterpolasi dan disimpan dalam variabel 'df_interpolated'.

Cuplikan 5 baris pertama dari data hasil interpolasi:
           t           x         y  crs       NO2
0 2020-10-23  112.760619 -7.223435  b''       NaN
1 2020-10-24  112.760619 -7.223435  b''  0.000034
2 2020-10-25  112.760619 -7.223435  b''  0.000012
3 2020-10-26  112.760619 -7.223435  b''  0.000076
4 2020-10-27  112.760619 -7.223435  b''  0.000141

Sisa missing values di kolom 'NO2': 1


In [6]:
df['t'] = pd.to_datetime(df['t'])
df_sorted = df.sort_values(by='t')
df_indexed = df_sorted.set_index('t')
df_indexed['NO2'] = df_indexed['NO2'].interpolate(method='time')
df_interpolated = df_indexed.reset_index()
print(f"Data diinterpolasi. Jumlah baris: {len(df_interpolated)}")

Data diinterpolasi. Jumlah baris: 1805


#### Analisis Nilai Outlier pada Data NO2

In [9]:
no2_values = df_interpolated['NO2']
q1 = no2_values.quantile(0.25)
q3 = no2_values.quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
outliers = df_interpolated[(no2_values < lower_bound) | (no2_values > upper_bound)]

# --- Konfirmasi Hasil ---
print(f"Selesai! Ditemukan {len(outliers)} nilai outlier pada data NO2.")
# Imputasi Nilai Outlier dengan Metode Median
df_no_outliers = df_interpolated.copy()
median_no2 = df_no_outliers['NO2'].median()
df_no_outliers.loc[(df_no_outliers['NO2'] < lower_bound) | (df_no_outliers['NO2'] > upper_bound), 'NO2'] = median_no2
print("Selesai! Nilai outlier telah diimputasi dengan median NO2.")

Selesai! Ditemukan 69 nilai outlier pada data NO2.
Selesai! Nilai outlier telah diimputasi dengan median NO2.


#### Ubah data time series ke data supervised

In [11]:
# Mulai dengan DataFrame 'df_interpolated' dari langkah sebelumnya
# 1. Pilih hanya kolom waktu ('t') dan nilai ('NO2')
data = df_interpolated[['t', 'NO2']].copy()

# 2. Tentukan berapa banyak lag yang ingin Anda buat (misalnya: 3 hari sebelumnya)
N_LAGS = 3

# 3. Buat fitur lag (geser data ke bawah)
for i in range(1, N_LAGS + 1):
    data[f'NO2(t-{i})'] = data['NO2'].shift(i)

# 4. Ganti nama kolom 'NO2' asli menjadi target (y)
data.rename(columns={'NO2': 'NO2_target(t)'}, inplace=True)

# 5. Hapus baris yang mengandung NaN
# Ini adalah N_LAGS baris pertama, yang tidak memiliki data historis
data.dropna(inplace=True)

# 6. Reset index agar rapi
data.reset_index(drop=True, inplace=True)

# 7. Simpan hasil akhir ke variabel baru 'df_supervised'
df_supervised = data

# 8. Tampilkan hasilnya
print(f"Selesai! DataFrame 'df_supervised' telah dibuat dengan {N_LAGS} lag features.")
print("\nCuplikan 5 baris pertama dari data supervised:")
print(df_supervised.head())

Selesai! DataFrame 'df_supervised' telah dibuat dengan 3 lag features.

Cuplikan 5 baris pertama dari data supervised:
           t  NO2_target(t)  NO2(t-1)  NO2(t-2)  NO2(t-3)
0 2020-10-27       0.000141  0.000076  0.000012  0.000034
1 2020-10-28       0.000021  0.000141  0.000076  0.000012
2 2020-10-29       0.000027  0.000021  0.000141  0.000076
3 2020-10-30       0.000033  0.000027  0.000021  0.000141
4 2020-10-31       0.000039  0.000033  0.000027  0.000021


#### AutoKorelasi & Seleksi Fitur
Cell ini menganalisis korelasi antara semua fitur lag (t-1 s/d t-14) dengan nilai target (t).
Fitur yang memiliki korelasi absolut di atas 0.5 akan dipilih untuk pemodelan.
Ini membantu mengurangi dimensi data dan hanya menggunakan fitur yang paling relevan.

In [17]:
# Mulai dengan DataFrame 'df_interpolated' dari langkah sebelumnya
data = df_interpolated[['t', 'NO2']].copy()
# Tentukan berapa banyak lag yang ingin Anda buat (misalnya: 14 hari)
N_LAGS_MAX = 14 
# Buat fitur lag (geser data ke bawah)
for i in range(1, N_LAGS_MAX + 1):
    data[f'NO2(t-{i})'] = data['NO2'].shift(i)
# Ganti nama kolom 'NO2' asli menjadi target (y)
data.rename(columns={'NO2': 'NO2_target(t)'}, inplace=True)
# Hapus baris yang mengandung NaN
data.dropna(inplace=True)
# Reset index agar rapi
data.reset_index(drop=True, inplace=True)
# Simpan hasil akhir ke variabel baru 'df_supervised_full'
df_supervised_full = data
print("\nMemulai Autokorelasi untuk Seleksi Fitur...")
# 1. Hitung matriks korelasi
corr_matrix = df_supervised_full.corr(numeric_only=True)
# 2. Ambil korelasi semua fitur dengan target
corr_target = corr_matrix['NO2_target(t)']
# 3. Pilih fitur lag yang memiliki korelasi absolut > 0.5
feature_cols = corr_target[(np.abs(corr_target) > 0.5) & (corr_target.index.str.startswith('NO2(t-'))].index.tolist()
print(f"Fitur lag terpilih (korelasi > 0.5): {feature_cols}")
# 4. Fallback jika tidak ada fitur yang lolos seleksi
if not feature_cols:
    print("PERINGATAN: Tidak ada fitur lag > 0.5. Menggunakan lag t-1 sebagai default.")
    feature_cols = ['NO2(t-1)'] # Fallback
# 5. Siapkan daftar kolom final untuk df_supervised
target_col = ['NO2_target(t)']
all_selected_cols = ['t'] + feature_cols + target_col
# 6. Buat DataFrame df_supervised yang baru HANYA dengan fitur terpilih
df_supervised = df_supervised_full[all_selected_cols].copy()
print(f"Kolom yang tidak terpilih telah di-drop.")
print("\nCuplikan 5 baris pertama dari df_supervised (setelah seleksi fitur):")
print(df_supervised.head())


Memulai Autokorelasi untuk Seleksi Fitur...
Fitur lag terpilih (korelasi > 0.5): ['NO2(t-1)', 'NO2(t-2)', 'NO2(t-3)']
Kolom yang tidak terpilih telah di-drop.

Cuplikan 5 baris pertama dari df_supervised (setelah seleksi fitur):
           t  NO2(t-1)  NO2(t-2)  NO2(t-3)  NO2_target(t)
0 2020-11-07  0.000020  0.000025  0.000030       0.000032
1 2020-11-08  0.000032  0.000020  0.000025       0.000030
2 2020-11-09  0.000030  0.000032  0.000020       0.000030
3 2020-11-10  0.000030  0.000030  0.000032       0.000026
4 2020-11-11  0.000026  0.000030  0.000030       0.000024


### Normalisasi Data (Z-Score)
Normalisasi data dilakukan menggunakan StandardScaler (Z-score). Tujuannya adalah untuk mengubah skala semua fitur sehingga memiliki rata-rata 0 dan standar deviasi 1. Ini penting untuk meningkatkan performa model K-NN yang sensitif terhadap skala data.

In [18]:
from sklearn.preprocessing import StandardScaler

# Pisahkan kolom fitur (X) dan target (y)
features_cols = [col for col in df_supervised.columns if col.startswith('NO2(t-')]
target_col = ['NO2_target(t)']

# Buat scaler terpisah untuk fitur dan target
feature_scaler_z = StandardScaler()
target_scaler_z = StandardScaler()

# Buat salinan DataFrame
df_scaled_z = df_supervised.copy()

# Fit dan transform fitur
df_scaled_z[features_cols] = feature_scaler_z.fit_transform(df_scaled_z[features_cols])
# Fit dan transform target
df_scaled_z[target_col] = target_scaler_z.fit_transform(df_scaled_z[target_col])

print(df_scaled_z.head())

           t  NO2(t-1)  NO2(t-2)  NO2(t-3)  NO2_target(t)
0 2020-11-07 -0.906693 -0.747785 -0.589003      -0.543413
1 2020-11-08 -0.543274 -0.906681 -0.747907      -0.593825
2 2020-11-09 -0.593682 -0.543263 -0.906811      -0.600431
3 2020-11-10 -0.600288 -0.593671 -0.543374      -0.712232
4 2020-11-11 -0.712079 -0.600277 -0.593785      -0.787335


## Model K-NN

In [19]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import time

#### Explorasi Data

In [21]:
print("--- 1. Informasi Tipe Data & Non-Null ---")
# .info() akan mencetak ringkasan DataFrame
df_scaled_z.info()
print("\n--- 2. Deskripsi Statistik ---")
# .describe() akan memberikan statistik deskriptif
print(df_scaled_z.describe())

print("\n--- 3. Pengecekan Missing Values ---")
# .isna().sum() akan menghitung jumlah nilai NaN di setiap kolom
print(df_scaled_z.isna().sum())

--- 1. Informasi Tipe Data & Non-Null ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1790 entries, 0 to 1789
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   t              1790 non-null   datetime64[ns]
 1   NO2(t-1)       1790 non-null   float64       
 2   NO2(t-2)       1790 non-null   float64       
 3   NO2(t-3)       1790 non-null   float64       
 4   NO2_target(t)  1790 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 70.1 KB

--- 2. Deskripsi Statistik ---
                                   t      NO2(t-1)      NO2(t-2)  \
count                           1790  1.790000e+03  1.790000e+03   
mean   2023-05-04 17:27:25.139664896  7.145122e-17 -7.939025e-18   
min              2020-11-07 00:00:00 -1.296698e+00 -1.296683e+00   
25%              2022-02-08 06:00:00 -6.985697e-01 -6.985584e-01   
50%              2023-05-05 12:00:00 -3.206834e-01 -3.206742e-0

#### Persiapan Data untuk Modelling

In [22]:
# Kita menggunakan DataFrame 'df_scaled_z' (hasil Z-Score)

# 1. Identifikasi Kolom Fitur (X)
# Ini adalah variabel prediktor (data historis)
feature_columns = [col for col in df_scaled_z.columns if col.startswith('NO2(t-')]

# 2. Identifikasi Kolom Target (y)
# Ini adalah variabel yang ingin diprediksi (data saat ini)
target_column = 'NO2_target(t)'

# 3. Buat variabel X dan y
# X (biasanya huruf besar) adalah DataFrame yang berisi semua fitur
X = df_scaled_z[feature_columns]

# y (biasanya huruf kecil) adalah Series yang berisi target
y = df_scaled_z[target_column]

# 4. Tampilkan konfirmasi
print("--- Variabel Fitur (X) ---")
print(f"Kolom yang digunakan: {feature_columns}")
print(X.head())

print("\n--- Variabel Target (y) ---")
print(f"Kolom yang digunakan: {target_column}")
print(y.head())

--- Variabel Fitur (X) ---
Kolom yang digunakan: ['NO2(t-1)', 'NO2(t-2)', 'NO2(t-3)']
   NO2(t-1)  NO2(t-2)  NO2(t-3)
0 -0.906693 -0.747785 -0.589003
1 -0.543274 -0.906681 -0.747907
2 -0.593682 -0.543263 -0.906811
3 -0.600288 -0.593671 -0.543374
4 -0.712079 -0.600277 -0.593785

--- Variabel Target (y) ---
Kolom yang digunakan: NO2_target(t)
0   -0.543413
1   -0.593825
2   -0.600431
3   -0.712232
4   -0.787335
Name: NO2_target(t), dtype: float64


#### Bagi data Training dan Testing

In [24]:
# Menggunakan X dan y dari langkah sebelumnya
train_size = int(len(X) * 0.8)

# 2. Bagi data X (fitur)
X_train = X.iloc[0:train_size]
X_test = X.iloc[train_size:len(X)]

# 3. Bagi data y (target)
y_train = y.iloc[0:train_size]
y_test = y.iloc[train_size:len(y)]

t_test = df_supervised['t'].iloc[train_size:len(df_supervised)]

# 4. Tampilkan konfirmasi
print(f"Total data: {len(X)} baris")
print(f"Jumlah data Latih (Train): {len(X_train)} baris (80%)")
print(f"Jumlah data Uji (Test): {len(X_test)} baris (20%)")


Total data: 1790 baris
Jumlah data Latih (Train): 1432 baris (80%)
Jumlah data Uji (Test): 358 baris (20%)


#### Hyperparameter Tuning (Mencari K Optimal)

Tahap ini melakukan iterasi untuk melatih dan mengevaluasi model K-NN dengan nilai 'K' yang berbeda (dari 1 hingga 30). Tujuannya adalah untuk menemukan nilai `best_k` (K optimal) yang menghasilkan RMSE terendah.

In [26]:
print("\nMemulai pencarian K optimal...")

# Persiapan: Ubah y_test ke skala asli untuk perbandingan RMSE
# Variabel target_scaler_z dan y_test sudah ada dari sel sebelumnya
y_test_orig = target_scaler_z.inverse_transform(y_test.values.reshape(-1, 1))

rmse_values = [] # Untuk menyimpan nilai RMSE
range_k = range(1, 31) # Kita akan tes K dari 1 sampai 30

# Loop untuk mencari K
for k in range_k:
    # 1. Latih model K-NN sementara
    knn_model_tune = KNeighborsRegressor(n_neighbors=k, n_jobs=-1)
    knn_model_tune.fit(X_train, y_train)
    
    # 2. Prediksi data test
    y_pred_tune = knn_model_tune.predict(X_test)
    
    # 3. Kembalikan prediksi ke skala asli
    y_pred_orig_tune = target_scaler_z.inverse_transform(y_pred_tune.reshape(-1, 1))
    
    # 4. Hitung RMSE
    rmse = np.sqrt(mean_squared_error(y_test_orig, y_pred_orig_tune))
    
    # 5. Simpan RMSE
    rmse_values.append(rmse)
    print(f"k = {k}, RMSE = {rmse}") # Cetak progres seperti di sel [40] Anda

# Cari K Terbaik
best_rmse_from_tuning = min(rmse_values)
# +1 karena index list mulai dari 0, tapi K kita mulai dari 1
best_k = rmse_values.index(best_rmse_from_tuning) + 1

print(f"\nPencarian K selesai. K optimal ditemukan: {best_k}")
print(f"Dengan nilai RMSE terendah: {best_rmse_from_tuning}")


Memulai pencarian K optimal...
k = 1, RMSE = 2.9298093726231992e-05
k = 2, RMSE = 2.1447579321554876e-05
k = 3, RMSE = 1.9441514891001982e-05
k = 4, RMSE = 1.8441144625944083e-05
k = 5, RMSE = 1.7533753994127963e-05
k = 6, RMSE = 1.6941059456630807e-05
k = 7, RMSE = 1.667804526955685e-05
k = 8, RMSE = 1.6435042535842854e-05
k = 9, RMSE = 1.6124034776027293e-05
k = 10, RMSE = 1.6258894822818882e-05
k = 11, RMSE = 1.5888261535517677e-05
k = 12, RMSE = 1.5884740689992907e-05
k = 13, RMSE = 1.5897922362492306e-05
k = 14, RMSE = 1.5851948601153985e-05
k = 15, RMSE = 1.5688005393794294e-05
k = 16, RMSE = 1.566383918986877e-05
k = 17, RMSE = 1.5506889948284097e-05
k = 18, RMSE = 1.5435961595274344e-05
k = 19, RMSE = 1.535753593705555e-05
k = 20, RMSE = 1.527293189243738e-05
k = 21, RMSE = 1.5222436641719718e-05
k = 22, RMSE = 1.531884910982575e-05
k = 23, RMSE = 1.5365719855578842e-05
k = 24, RMSE = 1.5360798304990498e-05
k = 25, RMSE = 1.5359792224693134e-05
k = 26, RMSE = 1.528783762568915

### Pelatihan Model K-NN
Cell ini menginisialisasi model K-NN Regressor dan melatihnya menggunakan data latih (X_train dan y_train). Proses .fit() adalah saat model mempelajari pola dari data.

In [31]:
# --- 2. Training Model kNN Optimal ---
# Menggunakan best_k yang ditemukan dari sel Hyperparameter Tuning
print(f"Melatih model kNN optimal (K={best_k}) dengan metrik 'euclidean'...")

# Inisialisasi dan latih model final
# Kita tetap gunakan 'knn_model' agar sel [36] dan [37] tetap berfungsi
knn_model = KNeighborsRegressor(
    n_neighbors=best_k, 
    metric='euclidean',  # Modifikasi: sebutkan metrik secara eksplisit
    n_jobs=-1
)

# Latih model
knn_model.fit(X_train, y_train)

print("Selesai! Model final disimpan dalam variabel 'knn_model'.")

Melatih model kNN optimal (K=30) dengan metrik 'euclidean'...
Selesai! Model final disimpan dalam variabel 'knn_model'.


### Pelaksanaan Prediksi
Model yang telah dilatih digunakan untuk membuat prediksi pada data uji (X_test). Hasil prediksi dari model disimpan dalam variabel y_pred.

In [32]:
y_pred = knn_model.predict(X_test)

# --- 5. Tampilkan Hasil ---
print("\nSelesai! Prediksi data test telah dibuat.")
print("Hasil prediksi disimpan dalam variabel 'y_pred'.")
print("\nCuplikan 5 hasil prediksi pertama (y_pred):")
print(y_pred[0:5])


Selesai! Prediksi data test telah dibuat.
Hasil prediksi disimpan dalam variabel 'y_pred'.

Cuplikan 5 hasil prediksi pertama (y_pred):
[-0.20825583  0.54852096 -0.48078236 -0.68209865 -0.2683705 ]


### Evaluasi Model
Cell ini bertujuan untuk mengukur performa model prediksi. Hasil prediksi (y_pred_orig) dibandingkan dengan data aktual (y_test_orig) menggunakan beberapa metrik evaluasi: MAE, MSE, dan RMSE, MAPE. Nilai metrik yang lebih rendah mengindikasikan tingkat akurasi model yang lebih tinggi.

In [35]:
# Import library baru untuk MAPE
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Mengembalikan nilai ke skala asli (unit NO2)
# y_pred diambil dari sel [36]
# y_test diambil dari sel [34]
# target_scaler_z diambil dari sel [30]
y_test_reshaped = y_test.values.reshape(-1, 1)
y_pred_reshaped = y_pred.reshape(-1, 1)

y_test_orig = target_scaler_z.inverse_transform(y_test_reshaped)
y_pred_orig = target_scaler_z.inverse_transform(y_pred_reshaped)

# Hitung Metrik Evaluasi
mae = mean_absolute_error(y_test_orig, y_pred_orig)
mse = mean_squared_error(y_test_orig, y_pred_orig)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test_orig, y_pred_orig) * 100 # MAPE BARU

# --- TAMPILKAN OUTPUT ---
# best_k diambil dari sel tuning yang baru Anda tambahkan
print(f"--- Evaluasi Model Optimal (K={best_k}) ---")
print(f"MAE:  {mae:.10f}")  # Format berbeda dari teman Anda
print(f"MSE:  {mse:.10f}")
print(f"RMSE: {rmse:.10f}")
print(f"MAPE: {mape:.2f}%") # Menambahkan MAPE

# Menambahkan blok if/else
if mape < 10:
    print("\nKualitas Model: Baik Sekali (MAPE < 10%)")
elif mape < 20:
    print("\nKualitas Model: Baik (MAPE 10-20%)")
else:
    print("\nKualitas Model: Cukup (MAPE > 20%)")

--- Evaluasi Model Optimal (K=30) ---
MAE:  0.0000093526
MSE:  0.0000000002
RMSE: 0.0000152088
MAPE: 34.20%

Kualitas Model: Cukup (MAPE > 20%)


#### Klasifikasi Indikator Kualitas Udara

Setelah model dievaluasi, kita dapat mengklasifikasikan nilai prediksi mentah (konsentrasi NO2) ke dalam kategori yang mudah dipahami (Sangat Rendah, Rendah, Sedang, Tinggi).

Penting: *Threshold* (ambang batas) untuk kategori ini dihitung menggunakan kuartil (Q1, Q2, Q3) **hanya dari data latih (Train)**. Ini untuk mencegah *data leakage*, yaitu agar model uji (Test) tidak memengaruhi cara kita membuat kategori.

In [38]:
import pandas as pd

print("\nMembuat klasifikasi 4-kategori (berbasis kuartil data latih)...")

# --- 1. Hitung Threshold Kuartil dari Data Latih ---
# Variabel df_supervised (un-normalized) ada dari sel [29]
# Variabel train_size (panjang data latih) ada dari sel [34]

# Ambil data target (y) yang TIDAK ternormalisasi dari data latih
train_data_orig_target = df_supervised['NO2_target(t)'].iloc[0:train_size]

# Hitung threshold kuartil
q1 = train_data_orig_target.quantile(0.25)
q2_median = train_data_orig_target.quantile(0.50)
q3 = train_data_orig_target.quantile(0.75)
print(f"Thresholds (dari data latih): Q1 (Sangat Rendah) <= {q1:.6f}, Q2 (Rendah) <= {q2_median:.6f}, Q3 (Sedang) <= {q3:.6f}")

# --- 2. Fungsi Klasifikasi ---
def classify_air_quality(value, q1, q2, q3):
    if value <= q1:
        return "Sangat Rendah"
    elif value <= q2:
        return "Rendah"
    elif value <= q3:
        return "Sedang"
    else:
        return "Tinggi"

# --- 3. Buat DataFrame Hasil ---
# Variabel t_test, y_test_orig, dan y_pred_orig sudah ada dari sel-sel sebelumnya
df_results = pd.DataFrame({
    'Tanggal': t_test.reset_index(drop=True), # Reset index t_test agar sesuai
    'Aktual': y_test_orig.flatten(),
    'Prediksi': y_pred_orig.flatten() # Menggunakan y_pred_orig dari sel [37]
})

# --- 4. Terapkan Klasifikasi ---
df_results['Kualitas Udara Aktual'] = df_results['Aktual'].apply(classify_air_quality, args=(q1, q2_median, q3))
df_results['Kualitas Udara Prediksi'] = df_results['Prediksi'].apply(classify_air_quality, args=(q1, q2_median, q3))

print("\nCuplikan 5 baris pertama hasil klasifikasi:")
print(df_results.head())

# Simpan ke CSV
df_results.to_csv('./dataset/hasil_prediksi_dan_klasifikasi.csv', index=False)
print("\nHasil lengkap disimpan di 'hasil_prediksi_dan_klasifikasi.csv'")


Membuat klasifikasi 4-kategori (berbasis kuartil data latih)...
Thresholds (dari data latih): Q1 (Sangat Rendah) <= 0.000027, Q2 (Rendah) <= 0.000039, Q3 (Sedang) <= 0.000064

Cuplikan 5 baris pertama hasil klasifikasi:
     Tanggal    Aktual  Prediksi Kualitas Udara Aktual Kualitas Udara Prediksi
0 2024-10-28  0.000073  0.000042                Tinggi                  Sedang
1 2024-10-29  0.000014  0.000066         Sangat Rendah                  Tinggi
2 2024-10-30  0.000008  0.000034         Sangat Rendah                  Rendah
3 2024-10-31  0.000053  0.000027                Sedang                  Rendah
4 2024-11-01  0.000034  0.000040                Rendah                  Sedang

Hasil lengkap disimpan di 'hasil_prediksi_dan_klasifikasi.csv'
